In [0]:
# ライブラリのインストール
%pip install altair
%pip install vega-datasets

Python interpreter will be restarted.
Requirement already satisfied: altair in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (4.1.0)
Requirement already satisfied: entrypoints in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (0.3)
Requirement already satisfied: pandas>=0.18 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (1.1.3)
Requirement already satisfied: jsonschema in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (3.2.0)
Requirement already satisfied: jinja2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (2.11.2)
Requirement already satisfied: numpy in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (1.19.2)
Requirement already satisfied: toolz in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from altair) (0.11.1)
Requirement already satisfied: python-dateutil>=2.7.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from pandas>=0.18->altair) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from pandas>=0.18->altair) (2020.5)
Requirement already satisfied: pyrsistent>=0.14.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from jsonschema->altair) (0.17.3)
Requirement already satisfied: six>=1.11.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from jsonschema->altair) (1.15.0)
Requirement already satisfied: attrs>=17.4.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from jsonschema->altair) (20.3.0)
Requirement already satisfied: setuptools in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from jsonschema->altair) (50.3.1.post20201107)
Requirement already satisfied: MarkupSafe>=0.23 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from jinja2->altair) (1.1.1)
Python interpreter will be restarted.
Python interpreter will be restarted.
Collecting vega-datasets
 Downloading vega_datasets-0.9.0-py3-none-any.whl (210 kB)
Requirement already satisfied: pandas in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from vega-datasets) (1.1.3)
Requirement already satisfied: pytz>=2017.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from pandas->vega-datasets) (2020.5)
Requirement already satisfied: numpy>=1.15.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from pandas->vega-datasets) (1.19.2)
Requirement already satisfied: python-dateutil>=2.7.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from pandas->vega-datasets) (2.8.1)
Requirement already satisfied: six>=1.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-1b98c05b-64fb-440c-84bc-3abc12f14275/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas->vega-datasets) (1.15.0)
Installing collected packages: vega-datasets
Successfully installed vega-datasets-0.9.0
Python interpreter will be restarted.

# 4. New York Times COVID-19 データセット分析

このノートブックは[New York Times COVID-19 データセット](https://github.com/nytimes/covid-19-data)に対する簡単な処理、分析を行うためのものです。データは定期的に`/databricks-datasets/COVID/covid-19-data/`で更新されるので、直接データにアクセスすることができます。

In [0]:
# ユーザーごとに一意のパスになるようにユーザー名をパスに含めます
import re
from pyspark.sql.types import * 
import os

# Username を取得
username_raw = dbutils.notebook.entry_point.getDbutils().notebook().getContext().tags().apply('user')
# Username の英数字以外を除去し、全て小文字化。Username をファイルパスやデータベース名の一部で使用可能にするため。
username = re.sub('[^A-Za-z0-9]+', '', username_raw).lower()

print(username)

os.environ['username']=''.join(username)

takaakiyayoidatabrickscom

In [0]:
# 標準ライブラリ
import io

# 外部ライブラリ
import requests
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data

# 地形データ
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_king = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/WA/King.geo.json'

## 郡のFIPS(Federal Information Processing Series)のダウンロード、緯度経度への変換

[Understanding Geographic Identifiers \(GEOIDs\)](https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html)

In [0]:
%sh mkdir -p /dbfs/tmp/$username/COVID/map_fips/ && wget -O /dbfs/tmp/$username/COVID/map_fips/countyfips_lat_long.csv https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv && ls -al /dbfs/tmp/$username/COVID/map_fips/

--2021-05-27 03:36:24-- https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129178 (126K) [text/plain]
Saving to: ‘/dbfs/tmp/takaakiyayoidatabrickscom/COVID/map_fips/countyfips_lat_long.csv’

 0K .......... .......... .......... .......... .......... 39% 5.17M 0s
 50K .......... .......... .......... .......... .......... 79% 4.23M 0s
 100K .......... .......... ...... 100% 3.25M=0.03s

2021-05-27 03:36:24 (4.27 MB/s) - ‘/dbfs/tmp/takaakiyayoidatabrickscom/COVID/map_fips/countyfips_lat_long.csv’ saved [129178/129178]

total 135
drwxrwxrwx 2 root root 4096 May 27 2021 .
drwxrwxrwx 2 root root 4096 May 27 03:36 ..
-rwxrwxrwx 1 root root 129178 May 27 2021 countyfips_lat_long.csv

In [0]:
# 郡のFIPSと中心の緯度経度のマッピングの作成
map_fips = spark.read.option("header", True).option("inferSchema", True).csv(f"/tmp/{username}/COVID/map_fips/countyfips_lat_long.csv")
map_fips = (map_fips
              .withColumnRenamed("STATE", "state")
              .withColumnRenamed("COUNTYNAME", "county")
              .withColumnRenamed("LAT", "lat")
              .withColumnRenamed("LON", "long_"))
map_fips.createOrReplaceTempView("map_fips")

In [0]:
map_fips_dedup = spark.sql("""select fips, min(state) as state, min(county) as county, min(long_) as long_, min(lat) as lat from map_fips group by fips""")
map_fips_dedup.createOrReplaceTempView("map_fips_dedup")

## 2019年の人口推定値のダウンロード

In [0]:
%sh mkdir -p /dbfs/tmp/$username/COVID/population_estimates_by_county/ && wget -O /dbfs/tmp/$username/COVID/population_estimates_by_county/co-est2019-alldata.csv https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/co-est2019-alldata.csv && ls -al /dbfs/tmp/$username/COVID/population_estimates_by_county/

--2021-05-27 03:40:52-- https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/co-est2019-alldata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3644730 (3.5M) [text/plain]
Saving to: ‘/dbfs/tmp/takaakiyayoidatabrickscom/COVID/population_estimates_by_county/co-est2019-alldata.csv’

 0K .......... .......... .......... .......... .......... 1% 5.10M 1s
 50K .......... .......... .......... .......... .......... 2% 4.55M 1s
 100K .......... .......... .......... .......... .......... 4% 4.02M 1s
 150K .......... .......... .......... .......... .......... 5% 4.15M 1s
 200K .......... .......... .......... .......... .......... 7% 3.25M 1s
 250K .......... .......... .......... .......... .......... 8% 5.00M 1s
 300K .......... .......... .......... .......... .......... 9% 3.96M 1s
 350K .......... .......... .......... .......... .......... 11% 8.67M 1s
 400K .......... .......... .......... .......... .......... 12% 10.1M 1s
 450K .......... .......... .......... .......... .......... 14% 9.82M 1s
 500K .......... .......... .......... .......... .......... 15% 25.2M 1s
 550K .......... .......... .......... .......... .......... 16% 17.7M 1s
 600K .......... .......... .......... .......... .......... 18% 23.4M 0s
 650K .......... .......... .......... .......... .......... 19% 26.5M 0s
 700K .......... .......... .......... .......... .......... 21% 19.5M 0s
 750K .......... .......... .......... .......... .......... 22% 24.5M 0s
 800K .......... .......... .......... .......... .......... 23% 21.2M 0s
 850K .......... .......... .......... .......... .......... 25% 22.5M 0s
 900K .......... .......... .......... .......... .......... 26% 25.0M 0s
 950K .......... .......... .......... .......... .......... 28% 18.4M 0s
 1000K .......... .......... .......... .......... .......... 29% 21.8M 0s
 1050K .......... .......... .......... .......... .......... 30% 23.5M 0s
 1100K .......... .......... .......... .......... .......... 32% 20.6M 0s
 1150K .......... .......... .......... .......... .......... 33% 23.3M 0s
 1200K .......... .......... .......... .......... .......... 35% 22.8M 0s
 1250K .......... .......... .......... .......... .......... 36% 19.3M 0s
 1300K .......... .......... .......... .......... .......... 37% 24.0M 0s
 1350K .......... .......... .......... .......... .......... 39% 23.6M 0s
 1400K .......... .......... .......... .......... .......... 40% 22.4M 0s
 1450K .......... .......... .......... .......... .......... 42% 22.4M 0s
 1500K .......... .......... .......... .......... .......... 43% 25.9M 0s
 1550K .......... .......... .......... .......... .......... 44% 22.2M 0s
 1600K .......... .......... .......... .......... .......... 46% 20.0M 0s
 1650K .......... .......... .......... .......... .......... 47% 21.8M 0s
 1700K .......... .......... .......... .......... .......... 49% 22.3M 0s
 1750K .......... .......... .......... .......... .......... 50% 24.8M 0s
 1800K .......... .......... .......... .......... .......... 51% 23.1M 0s
 1850K .......... .......... .......... .......... .......... 53% 21.8M 0s
 1900K .......... .......... .......... .......... .......... 54% 25.2M 0s
 1950K .......... .......... .......... .......... .......... 56% 24.9M 0s
 2000K .......... .......... .......... .......... .......... 57% 27.3M 0s
 2050K .......... .......... .......... .......... .......... 59% 21.0M 0s
 2100K .......... .......... .......... .......... .......... 60% 25.5M 0s
 2150K .......... .......... .......... .......... .......... 61% 22.9M 0s
 2200K .......... .......... .......... .......... .......... 63% 19.5M 0s
 2250K .......... .......... .......... .......... .......... 64% 24.6M 0s
 2300K .......... .......... .......... ...

In [0]:
map_popest_county = spark.read.option("header", True).option("inferSchema", True).csv(f"/tmp/{username}/COVID/population_estimates_by_county/co-est2019-alldata.csv")
map_popest_county.createOrReplaceTempView("map_popest_county")
fips_popest_county = spark.sql("select State * 1000 + substring(cast(1000 + County as string), 2, 3) as fips, STNAME, CTYNAME, census2010pop, POPESTIMATE2019 from map_popest_county")
fips_popest_county.createOrReplaceTempView("fips_popest_county")

## `nyt_daily`テーブルの作成
* ソース: `/databricks-datasets/COVID/covid-19-data/`
* 日次のCOVID-19レポートを格納

In [0]:
nyt_daily = spark.read.option("inferSchema", True).option("header", True).csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
nyt_daily.createOrReplaceTempView("nyt_daily")
display(nyt_daily)

date,county,state,fips,cases,deaths
2020-01-21,Snohomish,Washington,53061,1,0
2020-01-22,Snohomish,Washington,53061,1,0
2020-01-23,Snohomish,Washington,53061,1,0
2020-01-24,Cook,Illinois,17031,1,0
2020-01-24,Snohomish,Washington,53061,1,0
2020-01-25,Orange,California,6059,1,0
2020-01-25,Cook,Illinois,17031,1,0
2020-01-25,Snohomish,Washington,53061,1,0
2020-01-26,Maricopa,Arizona,4013,1,0
2020-01-26,Los Angeles,California,6037,1,0


## 特定の郡におけるCOVID-19感染者数、死者数

教育機関の閉鎖の時期における2週間枠に注目します。
* ワシントン州のトップ10の郡 (3/13/2020)
* ニューヨーク州のトップ10の郡 (3/18/2020)

クエリーでは、2019年の国勢調査の人口推定値を使用します。

In [0]:
# ワシントン州の2週間ウィンドウ
wa_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  from (
SELECT CAST(f.date AS date) AS date, f.county, f.fips, SUM(f.cases) AS cases, SUM(f.deaths) AS deaths, MAX(p.POPESTIMATE2019) AS population_estimate 
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = f.fips
 WHERE f.state = 'Washington' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, f.fips
) a""")
wa_state_window.createOrReplaceTempView("wa_state_window")

# ニューヨーク州の2週間ウィンドウ
ny_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-11T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-11T00:00:00.000+0000' AND '2020-03-25T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window.createOrReplaceTempView("ny_state_window")

# ニューヨーク州の2週間ウィンドウ (1週前)
ny_state_window_m1 = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window_m1.createOrReplaceTempView("ny_state_window_m1")

### ワシントン州、ニューヨーク州の郡におけるCOVID-19感染者数

In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-17,Pierce,44
2020-03-15,Whatcom,3
2020-03-15,Yakima,4
2020-03-09,Snohomish,45
2020-03-19,Spokane,9
2020-03-20,Pierce,83
2020-03-16,Whatcom,3
2020-03-12,Pierce,17
2020-03-12,Spokane,4
2020-03-19,King,693


In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-12,Westchester,147
2020-03-19,New York City,2477
2020-03-15,Albany,11
2020-03-19,Nassau,382
2020-03-13,Nassau,51
2020-03-14,Monroe,2
2020-03-19,Westchester,797
2020-03-23,Monroe,76
2020-03-25,Monroe,118
2020-03-12,Dutchess,1


In [0]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-07,Westchester,69
2020-03-12,Westchester,147
2020-03-19,New York City,2477
2020-03-15,Albany,11
2020-03-17,Saratoga,9
2020-03-19,Nassau,382
2020-03-13,Nassau,51
2020-03-14,Monroe,2
2020-03-19,Westchester,797
2020-03-12,Saratoga,3


## ワシントン州、ニューヨーク州の郡における10万人あたりのCOVID-19感染者数

上のグラフで注意すべきことは、感染者数の総数だけでは人口密度を考慮していないため、ワシントン州とニューヨーク州を比較するのは困難であるということです。完璧ではありませんが、事前の策として人口推定値に対する割合でデータを検証することにします。

人口に対するパーセンテージを見てみましょう。使用されている数字は2019年の国勢調査における郡ごとの人口推定値です。

**感染者のパーセンテージではなく、感染者数のトップ10の郡を検証していることに注意してください。**

In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10  
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-17,Pierce,4.86198590024
2020-03-15,Whatcom,1.30863217403
2020-03-15,Yakima,1.59443224261
2020-03-09,Snohomish,5.47389983736
2020-03-19,Spokane,1.72150620316
2020-03-20,Pierce,9.17147340273
2020-03-16,Whatcom,1.30863217403
2020-03-12,Pierce,1.87849455237
2020-03-12,Spokane,0.76511386807
2020-03-19,King,30.76196453984


In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-12,Westchester,15.19370422509
2020-03-19,New York City,152.08392429327
2020-03-15,Albany,3.60058394925
2020-03-19,Nassau,28.15190828668
2020-03-13,Nassau,3.75850084456
2020-03-14,Monroe,0.26962535557
2020-03-19,Westchester,82.37675011835
2020-03-23,Monroe,10.24576351160
2020-03-25,Monroe,15.90789597854
2020-03-12,Dutchess,0.33988403157


In [0]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-07,Westchester,7.13173871790
2020-03-12,Westchester,15.19370422509
2020-03-19,New York City,152.08392429327
2020-03-15,Albany,3.60058394925
2020-03-17,Saratoga,3.91537568030
2020-03-19,Nassau,28.15190828668
2020-03-13,Nassau,3.75850084456
2020-03-14,Monroe,0.26962535557
2020-03-19,Westchester,82.37675011835
2020-03-12,Saratoga,1.30512522677


## コロプレスマップによる州ごとの感染者数の可視化

郡の中心の緯度経度を取得するために`map_fips_dedup`とjoinします。

[Choropleth map \- Wikipedia](https://en.wikipedia.org/wiki/Choropleth_map#:~:text=A%20choropleth%20map%20(from%20Greek,each%20area%2C%20such%20as%20population)

In [0]:
# 日付番号と中心地の緯度経度の抽出
wa_daynum = spark.sql("""select f.fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from wa_state_window f join map_fips_dedup m on m.fips = f.fips""")
wa_daynum.createOrReplaceTempView("wa_daynum")
ny_daynum = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum.createOrReplaceTempView("ny_daynum")
ny_daynum_m1 = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window_m1 f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum_m1.createOrReplaceTempView("ny_daynum_m1")

In [0]:
# 地理情報の取得
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_ny = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json'
us_counties = alt.topo_feature(topo_usa, 'counties')
wa_counties = alt.topo_feature(topo_wa, 'cb_2015_washington_county_20m')
ny_counties = alt.topo_feature(topo_ny, 'cb_2015_new_york_county_20m')

In [0]:
# ワシントン州の検証
confirmed_wa = wa_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_wa['date'] = confirmed_wa['date'].astype(str)
deaths_wa = wa_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_wa['date'] = deaths_wa['date'].astype(str)

# ニューヨーク州の検証
confirmed_ny = ny_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny['date'] = confirmed_ny['date'].astype(str)
deaths_ny = ny_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny['date'] = deaths_ny['date'].astype(str)

# 一週前のニューヨーク州の検証
confirmed_ny_m1 = ny_daynum_m1.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny_m1['date'] = confirmed_ny_m1['date'].astype(str)
deaths_ny_m1 = ny_daynum_m1.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny_m1['date'] = deaths_ny_m1['date'].astype(str)

In [0]:
# 州ごとのコロプレスマップを可視化する関数
def map_state(curr_day_num, state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max):
  # date_strの取得
  date_str = confirmed[confirmed['day_num'] == 101]['date'].head(1).item()
  
  # 州
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # 郡
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
  ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(confirmed[(confirmed['confirmed_per100K'] > 0) & (confirmed['day_num'] == curr_day_num)], 'fips', ['confirmed_per100K', 'confirmed', 'county', 'date', 'fips'])  
  ).encode(
     color=alt.Color('confirmed_per100K:Q', scale=alt.Scale(type='log', domain=[confirmed_min, confirmed_max]), title='Confirmed per 100K'),
    tooltip=[
      alt.Tooltip('fips:O'),
      alt.Tooltip('confirmed:Q'),
      alt.Tooltip('confirmed_per100K:Q'),
      alt.Tooltip('county:N'),
      alt.Tooltip('date:N'),
    ],
  )

  # 緯度経度に基づく死者数
  points = alt.Chart(deaths[(deaths['deaths_per100K'] > 0) & (deaths['day_num'] == curr_day_num)]).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths_per100K):Q', scale=alt.Scale(type='symlog', domain=[deaths_min, deaths_max]), title='Deaths per 100K'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # 図のタイトルの更新
    title=f'COVID-19 {state_txt} Confirmed Cases and Deaths per 100K by County [{curr_day_num}, {date_str}]'
  )

  return (base_state + base_state_counties + points)

| 要因 | ワシントン | ニューヨーク | 
| ------- | -- | -- | 
| 教育機関の閉鎖| 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| 最大感染者数 | 794 | 20011 |
| 最大死者数 | 68 | 280 |
| 10万人あたり最大感染者数 | 50.55 | 1222.97 | 
| 10万人あたり最大死者数 | 3.27 | 17.11 |

### ワシントン州 (10万人あたりの感染者数、死者数)

In [0]:
map_state(101, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

Out[17]:

In [0]:
map_state(107, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

Out[18]:

In [0]:
map_state(114, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

Out[19]:

### ニューヨーク州 (10万人あたりの感染者数、死者数)

In [0]:
map_state(101, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

Out[20]:

In [0]:
map_state(107, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

Out[21]:

In [0]:
map_state(114, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

Out[22]:

## コロプレスマップによるワシントン州、ニューヨーク州の郡の感染者数、死者数の可視化(スライダー付き)

In [0]:
# 州のコロプレスマップの可視化関数
def map_state_slider(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  # day_numによる感染日のPivot
  confirmed_pv = confirmed[['fips', 'day_num', 'confirmed']].copy()
  confirmed_pv['fips'] = confirmed_pv['fips'].astype(str)
  confirmed_pv['day_num'] = confirmed_pv['day_num'].astype(str)
  confirmed_pv['confirmed'] = confirmed_pv['confirmed'].astype('int64')
  confirmed_pv = confirmed_pv.pivot_table(index='fips', columns='day_num', values='confirmed', fill_value=0).reset_index()

  # day_numによる死亡日のPivot
  deaths_pv = deaths[['lat', 'long_', 'day_num', 'deaths']].copy()
  deaths_pv['day_num'] = deaths_pv['day_num'].astype(str)
  deaths_pv['deaths'] = deaths_pv['deaths'].astype('int64')
  deaths_pv = deaths_pv.pivot_table(index=['lat', 'long_'], columns='day_num', values='deaths', fill_value=0).reset_index()

  # スライダーのカラム名の抽出
  column_names = confirmed_pv.columns.tolist()

  # 最初の要素の削除 (`fips`)
  column_names.pop(0)

  # intへの変換
  column_values = [None] * len(column_names)
  for i in range(0, len(column_names)): column_values[i] = int(column_names[i]) 
  
  # max_rowsの無効化
  alt.data_transformers.disable_max_rows()

  # 地理情報
  us_states = alt.topo_feature(topo_usa, 'states')
  us_counties = alt.topo_feature(topo_usa, 'counties')

  # 州、郡の境界線
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # スライダーの設定
  min_day_num = column_values[0]
  max_day_num = column_values[len(column_values)-1]
  slider = alt.binding_range(min=min_day_num, max=max_day_num, step=1)
  slider_selection = alt.selection_single(fields=['day_num'], bind=slider, name="day_num", init={'day_num':min_day_num})


  # 州ごとの感染者数
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
      stroke='black',
      strokeWidth=0.05
  ).transform_lookup(
      lookup='id',
      from_=alt.LookupData(confirmed_pv, 'fips', column_names)  
  ).transform_fold(
      column_names, as_=['day_num', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      day_num = 'parseInt(datum.day_num)',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
      color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).properties(
    # 図のタイトルの更新
    title=f'COVID-19 WA State Confirmed Cases by County'
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  ).transform_filter(
      slider_selection
  )

  # 緯度経度に基づく死者数
  points = alt.Chart(
    deaths_pv
  ).mark_point(
    opacity=0.75, filled=True
  ).transform_fold(
    column_names, as_=['day_num', 'deaths']
  ).transform_calculate(
      day_num = 'parseInt(datum.day_num)',
      deaths = 'isValid(datum.deaths) ? datum.deaths : -1'  
  ).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('deaths:Q', scale=alt.Scale(domain=(deaths_min, deaths_max), type='symlog'), title='deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
  ).add_selection(
      slider_selection
  ).transform_filter(
      slider_selection
  )

  # 感染者数(郡レベル)、死者数(点)
  return (base_state + base_state_counties + points) 

| 要因 | ワシントン | ニューヨーク | 
| ------- | -- | -- | 
| 教育機関の閉鎖| 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| 最大感染者数 | 794 | 20011 |
| 最大死者数 | 68 | 280 |
| 10万人あたり最大感染者数 | 50.55 | 1222.97 | 
| 10万人あたり最大死者数 | 3.27 | 17.11 |

In [0]:
map_state_slider('WA', wa_counties, confirmed_wa, 1, 800, deaths_wa, 1, 70, 53)

Out[24]:

In [0]:
map_state_slider('NY', ny_counties, confirmed_ny, 1, 21000, deaths_ny, 1, 300, 36)

Out[25]:

## ワシントン州、ニューヨーク州の郡におけるCOVID-19感染者数、死者数(地図、グラフ)

In [0]:
# map_state_graph
def map_state_graph(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  
  # 感染者数のpivot (日付による)
  confirmed_pv2 = confirmed[['fips', 'date', 'confirmed']].copy()
  confirmed_pv2['fips'] = confirmed_pv2['fips'].astype(str)
  confirmed_pv2['date'] = confirmed_pv2['date'].astype(str)
  confirmed_pv2['confirmed'] = confirmed_pv2['confirmed'].astype('int64')
  confirmed_pv2 = confirmed_pv2.pivot_table(index='fips', columns='date', values='confirmed', fill_value=0).reset_index()

  # 死者数のpivot
  deaths_pv2 = deaths[['lat', 'long_', 'date', 'deaths']].copy()
  deaths_pv2['date'] = deaths_pv2['date'].astype(str)
  deaths_pv2['deaths'] = deaths_pv2['deaths'].astype('int64')
  deaths_pv2 = deaths_pv2.pivot_table(index=['lat', 'long_'], columns='date', values='deaths', fill_value=0).reset_index()

  # スライドバーのためのカラム取得
  column_names2 = confirmed_pv2.columns.tolist()

  # 最初の要素の削除 (`fips`)
  column_names2.pop(0)

  # 日付の選択
  pts = alt.selection(type="single", encodings=['x'])

  # 州
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # 州、郡
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.05,
  ).transform_lookup(
    lookup='id',
   from_=alt.LookupData(confirmed_pv2, 'fips', column_names2)
   ).transform_fold(
     column_names2, as_=['date', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      date = 'datum.date',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
       color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).transform_filter(
    pts
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  )

  # 棒グラフ
  bar = alt.Chart(confirmed).mark_bar().encode(
      x='date:N',
      y='confirmed_per100K:Q',
      color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
  ).properties(
      width=800,
      height=200,
      title='Confirmed Cases per 100K'
  ).add_selection(pts)

  # 死者数
  points = alt.Chart(deaths).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths):Q', scale=alt.Scale(domain=[deaths_min, deaths_max]), title='Deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # 図のタイトルの更新
    title=f'COVID-19 Confirmed Cases and Deaths by County'
  ).transform_filter(
      pts
  )

  return (base_state + base_state_counties + points) & bar

In [0]:
map_state_graph('WA', wa_counties, confirmed_wa, 1, 800, deaths_wa, 1, 70, 53)

Out[27]:

In [0]:
map_state_graph('NY', ny_counties, confirmed_ny, 1, 21000, deaths_ny, 1, 300, 36)

Out[28]:

In [0]:
map_state_graph('NY', ny_counties, confirmed_ny_m1, 1, 4500, deaths_ny, 1, 70, 36)

Out[29]:

# END